#  ファインチューニング

## ライブラリーのインポート

In [1]:
import numpy as np
import random

import torch
import torch.nn  as nn
import torch.optim as optim 

from torchvision import models

from tqdm import tqdm

In [2]:
# 乱数のシードを固定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

## DatasetとDataLoaderを作成

In [4]:
# 1.3節で作成したクラスを同じフォルダにあるmake_dataset_dataloader.pyに記載して使用
from utils.dataloader_image_classification import ImageTransform, make_datapath_list, HymenopteraDataset

# アリとハチの画像へのファイルパスのリストを作成する
train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

# Datasetを作成する
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset = HymenopteraDataset(
    file_list=train_list, transform=ImageTransform(size, mean, std), phase='train')

val_dataset = HymenopteraDataset(
    file_list=val_list, transform=ImageTransform(size, mean, std), phase='val')

#DataLoaderを作成する
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(
    train_dataset , batch_size=batch_size , shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 辞書オブジェクトにまとめる
dataloarders_dict = {"ttain": train_dataloader , "val" : val_dataloader}

./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


## ネットワークモデルの作成

In [6]:
# 学習済のVGG-16モデルをロード

# VGG-16モデルのインスタンスを生成
use_pretrained =True #学習済のパラメータを使用
net = models.vgg16(pretrained = use_pretrained)

# VGG16の最後の出力層の出力ユニットをアリとハチの2つに付け替える
net.classifier[6]=nn.Linear(in_features=4096,out_features=2)

# 訓練モードに設定
net.train()

print('ネットワーク設定完了：学習済の重みをロードし、訓練モードに設定しました')


ネットワーク設定完了：学習済の重みをロードし、訓練モードに設定しました


## 損失関数を定義

In [7]:
# 損失関数の設定
criterion = nn.CrossEntropyLoss()

## 最適化手法を設定

In [10]:
#  ファインチューニングで学習させるパラメータを、変数params_to_updateの1〜3に格納する

params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# 学習させる層のパラメータ名を指定
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight" , 
                                                  "classifier.0.bias" , "classifier.3.weight" , "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight" , "classifier.6.biass"]

#  パラメータごとに各リストに格納する
for name , param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requiers_grad = True
        params_to_update_1.append(param)
        print("params_to_updata_1に格納：" , name)
        
    elif name in update_param_names_2:
        param.requiers_grad = True
        params_to_update_2.append(param)
        print("params_to_updata_2に格納：" , name)
    
    elif name in update_param_names_3:
        param.requiers_grad = True
        params_to_update_3.append(param)
        print("params_to_updata_3に格納：" , name)
        
    else:
        param.requires_grad = False
        print("勾配計算なし。学習しない：" , name)
    



params_to_updata_1に格納： features.0.weight
params_to_updata_1に格納： features.0.bias
params_to_updata_1に格納： features.2.weight
params_to_updata_1に格納： features.2.bias
params_to_updata_1に格納： features.5.weight
params_to_updata_1に格納： features.5.bias
params_to_updata_1に格納： features.7.weight
params_to_updata_1に格納： features.7.bias
params_to_updata_1に格納： features.10.weight
params_to_updata_1に格納： features.10.bias
params_to_updata_1に格納： features.12.weight
params_to_updata_1に格納： features.12.bias
params_to_updata_1に格納： features.14.weight
params_to_updata_1に格納： features.14.bias
params_to_updata_1に格納： features.17.weight
params_to_updata_1に格納： features.17.bias
params_to_updata_1に格納： features.19.weight
params_to_updata_1に格納： features.19.bias
params_to_updata_1に格納： features.21.weight
params_to_updata_1に格納： features.21.bias
params_to_updata_1に格納： features.24.weight
params_to_updata_1に格納： features.24.bias
params_to_updata_1に格納： features.26.weight
params_to_updata_1に格納： features.26.bias
params_to_updata_1に格納： f

In [12]:
#  最適化手法の設定
optimizer = optim.SGD([
    {'params': params_to_update_1 , 'lr' : 1e-4},
    {'params': params_to_update_2 , 'lr' : 5e-4},
    {'params': params_to_update_3 , 'lr' : 1e-3}  
] , momentum=0.9)

## 学習・検証の実施